In [1]:
import numpy as np
try:
    import rawpy
except ModuleNotFoundError:
    !pip3 install rawpy
    import rawpy

     |████████████████████████████████| 686kB 44.3MB/s 


In [0]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

In [0]:
input_dir = '/content/drive/My Drive/ImageDataset/Sony/Sony/short/'
gt_dir = '/content/drive/My Drive/ImageDataset/Sony/Sony/long/'
checkpoint_dir = '/content/drive/My Drive/CheckPoint2/'
result_dir = '/content/drive/My Drive/Results2/'

In [0]:
import glob
import os

In [0]:
test_fns = glob.glob(gt_dir + "1*.ARW")
test_ids = [int(os.path.basename(test_fn)[0:5]) for test_fn in test_fns]

In [0]:
ps = 512
save_freq = 50

In [0]:
DEBUG = 0
if DEBUG:
    save_freq = 2
    test_ids = test_ids[0:5]

In [0]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(out_channels),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(out_channels)
    )

In [0]:
def expansive_block(in_channels, mid_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(kernel_size=3, in_channels=in_channels, out_channels=mid_channels, padding=1),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(mid_channels),
        nn.Conv2d(kernel_size=3, in_channels=mid_channels, out_channels=mid_channels, padding=1),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(mid_channels),
        nn.ConvTranspose2d(in_channels=mid_channels, out_channels=out_channels, kernel_size=3, stride=2, padding=1, output_padding=1)
    )

In [0]:
def final_block(in_channels, mid_channels, out_channels, kernel_size=3):
    return nn.Sequential(
            nn.Conv2d(kernel_size=kernel_size, in_channels=in_channels, out_channels=mid_channels, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.BatchNorm2d(mid_channels),
            nn.Conv2d(kernel_size=kernel_size, in_channels=mid_channels, out_channels=mid_channels, padding=1),
            nn.LeakyReLU(inplace=True),
            nn.BatchNorm2d(mid_channels),
            nn.Conv2d(kernel_size=1, in_channels=mid_channels, out_channels=out_channels, padding=0),
            )

In [0]:
class DepthToSpace(nn.Module):

    def __init__(self, block_size):
        super().__init__()
        self.bs = block_size

    def forward(self, x):
        N, C, H, W = x.size()
        x = x.view(N, self.bs, self.bs, C // (self.bs ** 2), H, W)  # (N, bs, bs, C//bs^2, H, W)
        x = x.permute(0, 3, 4, 1, 5, 2).contiguous()  # (N, C//bs^2, H, bs, W, bs)
        x = x.view(N, C // (self.bs ** 2), H * self.bs, W * self.bs)  # (N, C//bs^2, H * bs, W * bs)
        # print("In DS shape "+str(x.shape))
        return x

In [0]:
def pack_raw(raw):
    #packing bayer sensor image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0)/(16383 - 512) #subtracting the black level
    # print(im.shape)
    im = np.expand_dims(im, axis=0)
    img_shape = im.shape
    # print(img_shape)
    H = img_shape[1]
    W = img_shape[2]
    # The "channels" channel might be needed to be moved ahead(in that case axis = 0)
    out = np.concatenate((im[:, 0:H:2, 0:W:2],
                          im[:, 0:H:2, 1:W:2],
                          im[:,1:H:2, 1:W:2],
                          im[:, 1:H:2, 0:W:2],
                          ), axis=0)
    return out

In [0]:
class UNet(nn.Module):
    def __init__(self, n_out_channels):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(4, 32)
        self.dconv_down2 = double_conv(32, 64)
        self.dconv_down3 = double_conv(64, 128)
        self.dconv_down4 = double_conv(128, 256)
        self.dconv_down5 = double_conv(256, 512)
        self.maxpool = nn.MaxPool2d(2)
        self.conv2d_t = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.dconv_up1 = expansive_block(in_channels = 512, mid_channels = 256, out_channels = 128)
        self.dconv_up2 = expansive_block(in_channels = 256, mid_channels = 128, out_channels = 64)
        self.dconv_up3 = expansive_block(in_channels = 128, mid_channels = 64, out_channels = 32)
        self.final_layer = final_block(in_channels = 64, mid_channels = 32, out_channels = n_out_channels)

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        # print("Con1= "+str(conv1.shape))
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        # print("Con2= "+str(conv2.shape))
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)
        # print("Con3 = "+ str(conv3.shape))
        conv4 = self.dconv_down4(x)
        x = self.maxpool(conv4)
        # print("Con4= "+str(conv4.shape))
        conv5 = self.dconv_down5(x)
        # print("Con5= "+str(conv5.shape))
        # First upsampling
        x = self.conv2d_t(conv5)
        # print("First upcon+conv4 = "+str(x.shape))
        #concatenation and up_conv(Includes 2 conv+up_conv)
       
        x = torch.cat([x, conv4], dim = 1)
        #256+256 channels as ip
        x = self.dconv_up1(x)
        # print("Second upcon+conv3 = "+str(x.shape))
        #has 128 channels op
        x = torch.cat([x, conv3], dim = 1)
        #128+128 channels as ip
        x = self.dconv_up2(x)
        # has 64 channels op
        # print("third upcon+conv2 = "+str(x.shape))
        x = torch.cat([x, conv2], dim = 1)
        #64+64 channels as ip
        x = self.dconv_up3(x)
        # has 32 channels op
        # print("fourth upcon+conv1 = "+str(x.shape))
        x = torch.cat([x, conv1], dim = 1)
        # has 32+32 channels
        x = self.final_layer(x)
        # print("pre final shape = ", str(x.shape))
        x = DepthToSpace(2)(x)
        # print("X shape = "+str(x.shape))
        return x

In [0]:
dirs = [checkpoint_dir+str(d) for d in os.listdir(checkpoint_dir)]
dirs  = sorted(dirs, reverse=True)
# dirs = sorted(dirs, key=lambda x: os.path.getctime(x), reverse=True)[:1]

In [0]:
from torchsummary import summary

In [0]:
net = UNet(12)

In [0]:
net = net.cuda()

In [14]:
summary(net, (4, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 512, 512]           1,184
         LeakyReLU-2         [-1, 32, 512, 512]               0
       BatchNorm2d-3         [-1, 32, 512, 512]              64
            Conv2d-4         [-1, 32, 512, 512]           9,248
         LeakyReLU-5         [-1, 32, 512, 512]               0
       BatchNorm2d-6         [-1, 32, 512, 512]              64
         MaxPool2d-7         [-1, 32, 256, 256]               0
            Conv2d-8         [-1, 64, 256, 256]          18,496
         LeakyReLU-9         [-1, 64, 256, 256]               0
      BatchNorm2d-10         [-1, 64, 256, 256]             128
           Conv2d-11         [-1, 64, 256, 256]          36,928
        LeakyReLU-12         [-1, 64, 256, 256]               0
      BatchNorm2d-13         [-1, 64, 256, 256]             128
        MaxPool2d-14         [-1, 64, 1

In [0]:
epoch = int(dirs[0][-15:-11])

In [31]:
epoch

337

In [0]:
if not os.path.isdir(result_dir+'final/'):
    os.makedirs(result_dir + 'final/')

In [0]:
dirs = [checkpoint_dir+str(d) for d in os.listdir(checkpoint_dir)]
dirs  = sorted(dirs, reverse=True)
# dirs = sorted(dirs, key=lambda x: os.path.getctime(x), reverse=True)[:1]

In [34]:
dirs

['/content/drive/My Drive/CheckPoint2/0337-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0336-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0335-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0334-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0333-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0332-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0331-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0330-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0329-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0328-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0327-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0326-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0325-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0324-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0323-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0322-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0321-model.ckpt',
 '/content/drive/My Drive/CheckPoint2/0320-model

In [0]:
if dirs == []:
    train_epoch = None
else:
    train_epoch = int(dirs[0][-15:-11])

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
net = UNet(12)
net = net.to(device)
# optimizer = optim.Adam(net.parameters(), lr = 8e-5)
if train_epoch != None:
    checkpoint = torch.load(checkpoint_dir+'/%04d-model.ckpt'%train_epoch)
    net.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    # train_epoch = checkpoint['epoch']
    # loss = checkpoint['loss']

In [0]:
def abs_L1_loss(output, target):
    G_loss = torch.mean(torch.abs(output - target))
    return G_loss

In [0]:
import cv2
from PIL import Image

In [40]:
for test_id in test_ids:
    in_files = glob.glob(input_dir + "%05d_00*.ARW" %test_id)
    for k in range(len(in_files)):
        in_path = in_files[k]
        in_fn = os.path.basename(in_path)
        gt_files = glob.glob(gt_dir + "%05d_00*.ARW"%test_id)
        gt_path = gt_files[0]
        gt_fn = os.path.basename(gt_path)
        in_exposure = float(in_fn[9:-5])
        gt_exposure = float(gt_fn[9:-5])
        ratio = min(gt_exposure/in_exposure, 300)
        
        raw = rawpy.imread(in_path)
        input_full = np.expand_dims(pack_raw(raw), axis = 0)*ratio
        input_full = np.minimum(input_full, 1.0)

        im = raw.postprocess(use_camera_wb = True, half_size = False, no_auto_bright = True, output_bps = 16)
        scale_full = np.expand_dims(np.float32(im/65535.0), axis = 0)

        # gt_im = cv2.imread(gt_path)
        gt_raw = rawpy.imread(gt_path)
        gt_im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
        gt_full = np.expand_dims(np.float32(gt_im/65535.0), axis=0)  
        gt_full = np.transpose(gt_full, (0, 3, 1, 2))

        input_full =torch.tensor(input_full, device=device).float()
        gt_full = torch.tensor(gt_full, device = device).float()
        out = net(input_full)

        out = out.detach().cpu().numpy()
        gt_full = gt_full.cpu().numpy()

        out = np.minimum(np.maximum(out, 0), 1)

        out = out[0,:,:,:]
        out = np.transpose(out, (1, 2, 0))
        gt_full = gt_full[0,:,:,:]
        gt_full = np.transpose(gt_full, (1, 2, 0))
        scale_full = scale_full[0,:,:,:]
        print("Out = "+str(out.shape))
        # print("gt = "+str(gt_full.shape))
        # print("scale = "+str(scale_full.shape))
        scale_full = scale_full*np.mean(gt_full)/np.mean(scale_full)
        # print(out[10]*255)
        Image.fromarray((out*255).astype(np.uint8)).save(
                result_dir + 'final/%05d_00_%d_out.jpg' % (test_id, ratio))
        Image.fromarray((scale_full*255).astype(np.uint8)).save(
                result_dir + 'final/%05d_00_%d_scale.jpg' % (test_id, ratio))
        Image.fromarray((gt_full*255).astype(np.uint8)).save(
                result_dir + 'final/%05d_00_%d_gt.jpg' % (test_id, ratio))

Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (2848, 4256, 3)
Out = (284